In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import mlflow
import mlflow.sklearn
import json
from pathlib import Path

# Создаём папку для отчётов если её нет
Path('../reports').mkdir(exist_ok=True)

print("Все библиотеки загружены успешно!")

Все библиотеки загружены успешно!


In [3]:
# Загружаем датасет
housing = fetch_california_housing()
X = pd.DataFrame(housing.data, columns=housing.feature_names)
y = pd.Series(housing.target, name='Price')

print(f"Размер датасета: {X.shape[0]} строк, {X.shape[1]} признаков")
print(f"\nСтатистика целевой переменной (цена):")
print(y.describe())
print(f"\nПервые 5 строк:")
print(X.head())

Размер датасета: 20640 строк, 8 признаков

Статистика целевой переменной (цена):
count    20640.000000
mean         2.068558
std          1.153956
min          0.149990
25%          1.196000
50%          1.797000
75%          2.647250
max          5.000010
Name: Price, dtype: float64

Первые 5 строк:
   MedInc  HouseAge  AveRooms  AveBedrms  Population  AveOccup  Latitude  \
0  8.3252      41.0  6.984127   1.023810       322.0  2.555556     37.88   
1  8.3014      21.0  6.238137   0.971880      2401.0  2.109842     37.86   
2  7.2574      52.0  8.288136   1.073446       496.0  2.802260     37.85   
3  5.6431      52.0  5.817352   1.073059       558.0  2.547945     37.85   
4  3.8462      52.0  6.281853   1.081081       565.0  2.181467     37.85   

   Longitude  
0    -122.23  
1    -122.22  
2    -122.24  
3    -122.25  
4    -122.25  


In [4]:
# Разделяем на train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Размер обучающей выборки: {X_train.shape[0]} строк")
print(f"Размер тестовой выборки: {X_test.shape[0]} строк")

Размер обучающей выборки: 16512 строк
Размер тестовой выборки: 4128 строк


In [5]:
# Настраиваем MLflow
mlflow.set_experiment("california_housing_experiment")

# Начинаем логирование
with mlflow.start_run(run_name="linear_regression_baseline"):
    
    # Обучаем модель
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    # Делаем предсказания
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    
    # Вычисляем метрики
    train_mse = mean_squared_error(y_train, y_pred_train)
    test_mse = mean_squared_error(y_test, y_pred_test)
    train_r2 = r2_score(y_train, y_pred_train)
    test_r2 = r2_score(y_test, y_pred_test)
    train_mae = mean_absolute_error(y_train, y_pred_train)
    test_mae = mean_absolute_error(y_test, y_pred_test)
    
    # Логируем гиперпараметры (их у LinearRegression нет, но логируем для примера)
    mlflow.log_param("model_type", "LinearRegression")
    mlflow.log_param("test_size", 0.2)
    
    # Логируем метрики
    mlflow.log_metric("train_mse", train_mse)
    mlflow.log_metric("test_mse", test_mse)
    mlflow.log_metric("train_r2", train_r2)
    mlflow.log_metric("test_r2", test_r2)
    mlflow.log_metric("train_mae", train_mae)
    mlflow.log_metric("test_mae", test_mae)
    
    # Логируем модель
    mlflow.sklearn.log_model(model, "model")
    
    # Сохраняем метрики в JSON для отчёта
    metrics_dict = {
        "train_mse": train_mse,
        "test_mse": test_mse,
        "train_r2": train_r2,
        "test_r2": test_r2,
        "train_mae": train_mae,
        "test_mae": test_mae
    }
    
    with open('../reports/metrics.json', 'w') as f:
        json.dump(metrics_dict, f, indent=2)
    
    print("Модель обучена и залогирована в MLflow!")
    print(f"\nМетрики:")
    print(f"  Train MSE: {train_mse:.4f}")
    print(f"  Test MSE: {test_mse:.4f}")
    print(f"  Train R²: {train_r2:.4f}")
    print(f"  Test R²: {test_r2:.4f}")

2025/11/23 00:15:19 INFO mlflow.tracking.fluent: Experiment with name 'california_housing_experiment' does not exist. Creating a new experiment.
2025/11/23 00:15:20 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/23 00:15:44 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Модель обучена и залогирована в MLflow!

Метрики:
  Train MSE: 0.5179
  Test MSE: 0.5559
  Train R²: 0.6126
  Test R²: 0.5758
